In [ ]:
pip install selenium

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# Setup WebDriver
options = Options()
options.add_argument("--start-maximized")  
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)

url = 'https://www.psx.com.pk/market-summary/'
driver.get(url)
time.sleep(3) 

# Slowly scroll to the bottom of the page
scroll_pause_time = 0.5
scroll_step = 300  # pixels per scroll
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    for i in range(0, last_height, scroll_step):
        driver.execute_script(f"window.scrollTo(0, {i});")
        time.sleep(scroll_pause_time)
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  
    last_height = new_height

print("✅ Finished scrolling")

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

table = soup.find('div', {'class': 'tab-content'})

rows = table.find_all('tr')
print(f"✅ Found {len(rows)-1} rows (excluding header)")

psx_data = []

for row in rows[1:]: 
    cols = row.find_all('td')
    if len(cols) >= 8:
        scrip = cols[0].text.strip()
        ldcp = cols[1].text.strip()
        open_ = cols[2].text.strip()
        high = cols[3].text.strip()
        low = cols[4].text.strip()
        current = cols[5].text.strip()
        change = cols[6].text.strip()
        volume = cols[7].text.strip()

        psx_data.append([
            scrip, ldcp, open_, high, low, current, change, volume
        ])

for stock in psx_data:
    print(stock) 

✅ Finished scrolling
✅ Found 641 rows (excluding header)
['SCRIP', 'LDCP', 'OPEN', 'HIGH', 'LOW', 'CURRENT', 'CHANGE', 'VOLUME']
['AL-Ghazi Tractors', '402.41', '386.0', '409.0', '370.0', '389.85', '-13.88', '31,919']
['Atlas Honda Ltd', '1,166.02', '1167.0', '1179.9', '1160.0', '1169.8', '3.15', '8,848']
['Dewan Motors', '32.75', '33.75', '36.03', '33.5', '36.03', '3.28', '5,440,689']
['Ghandhara Automobile', '478.59', '479.0', '486.0', '470.0', '476.0', '-5.78', '1,491,532']
['Ghandhara Ind.', '707.30', '708.0', '715.98', '689.0', '689.0', '-16.23', '663,397']
['Hinopak Motor', '342.24', '343.0', '345.0', '334.0', '334.0', '-7.58', '8,961']
['Honda Atlas Cars', '284.38', '287.0', '292.64', '283.01', '286.5', '1.72', '795,537']
['Indus Motor Co.XD', '1,935.55', '1935.0', '1948.0', '1930.0', '1930.0', '-4.74', '5,872']
['Millat Tractors', '588.19', '592.0', '592.0', '580.0', '586.0', '-4.86', '82,988']
['Sazgar Engineering', '1,318.54', '1322.86', '1340.0', '1306.0', '1313.0', '-4.25',

In [22]:
import csv
filename = "psx_data.csv"
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow([
      "scrip", "ldcp", "open_", "high", "low", "current", "change", "volume"
    ])
    writer.writerows(psx_data)

print(f"✅ Data saved to {filename}")

✅ Data saved to psx_data.csv
